In [1]:
import os
import time

path = "/workspace/AAIPL/hf_models/Qwen2.5-14B-Instruct"

print("Exists:", os.path.exists(path))
print("Files:", os.listdir(path))


Exists: True
Files: ['model-00008-of-00008.safetensors', 'model-00006-of-00008.safetensors', 'generation_config.json', 'model-00005-of-00008.safetensors', 'README.md', 'config.json', '.ipynb_checkpoints', 'model.safetensors.index.json', 'vocab.json', 'model-00003-of-00008.safetensors', 'LICENSE', 'merges.txt', 'model-00001-of-00008.safetensors', 'model-00007-of-00008.safetensors', 'model-00002-of-00008.safetensors', 'tokenizer.json', 'tokenizer_config.json', '.gitattributes', 'model-00004-of-00008.safetensors']


In [1]:
from unsloth import FastLanguageModel
import torch

model_path = "/workspace/AAIPL/outputs/final"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = 1024,          # safer first load
)

FastLanguageModel.for_inference(model)

print("Loaded successfully ✅")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 02-15 05:02:52 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 255.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - igno

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Unsloth 2025.10.9 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


Loaded successfully ✅


In [49]:
messages = [
    {
        "role": "system",
        "content": """
You are an expert logical reasoning generator.

Rules:
- Output valid JSON only
- Generate EXACTLY one question
- Difficulty MUST be VERY HARD
- Do NOT fabricate complexity
"""
    },
    {
        "role": "user",
        "content": f"""
Generate exactly ONE VERY HARD logical reasoning question.

Topic: Alphanumeric Series

Difficulty constraints:
- Minimum 6 terms
- Include alternating pattern logic
- Include nonlinear number changes
- Include irregular letter shifts
- Must require multi-step reasoning
- Must NOT be solvable by simple arithmetic progression
- Explanation must describe the TRUE underlying pattern

JSON format:
{{
    "topic": "Alphanumeric Series",
    "question": "<full question>",
    "choices": [
        "A) ...",
        "B) ...",
        "C) ...",
        "D) ..."
    ],
    "answer": "<letter>",
    "explanation": "<≤100 words>"
}}
"""
    }
]

In [50]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")


In [51]:
start_time = time.time()

# outputs = model.generate(
#     **inputs,
#     max_new_tokens=500,
#     temperature=0.7,
#     top_p=0.9,
#     do_sample=True,
# )

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 500,
    temperature = 0.7,
    top_p = 0.9,
    do_sample = True,
)


end_time = time.time()


In [52]:
response = tokenizer.decode(
    outputs[0][inputs.shape[1]:],   # ✅ slice आउट input हटाओ
    skip_special_tokens=True
)

print("Generated Output:\n")
print(response)

print("\n⏱ Time taken:", round(end_time - start_time, 3), "seconds")


Generated Output:

{
    "topic": "Alphanumeric Series",
    "question": "Identify the next term in the series: B2, D5, G9, K14, O20, ?",
    "choices": [
        "A) R27",
        "B) S28",
        "C) T30",
        "D) U32"
    ],
    "answer": "D",
    "explanation": "The series alternates between letter and number patterns. Letters advance by increasing intervals (1, 2, 3, 4), while numbers follow a quadratic increment (2, 5, 9, 14, 20). The next letter is U (+5 from O) and the next number is 32 (quadratic increment from 20)."
}

⏱ Time taken: 3.818 seconds
